1. Quyét dữ liệu inflation và tương đương

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
MacroE_url = 'https://tradingeconomics.com/vietnam/inflation-cpi'
response = requests.get(MacroE_url)
# allsoup = BeautifulSoup(allpage.text, 'html') 
# all_table = allsoup.find('tbody')
response

<Response [403]>

In [4]:
# Kiểm tra nếu yêu cầu thành công
if response.status_code == 200:
    # Sử dụng BeautifulSoup để phân tích cú pháp HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm bảng có class là "text-right text-nowrap" (hoặc class của bạn)
    table = soup.find('tbody')

    # Tạo danh sách trống để lưu dữ liệu
    data = []

    # Lặp qua các hàng trong bảng
    for row in table.find_all('tr'):
        # Lấy tất cả các cột trong mỗi hàng
        columns = row.find_all('td')
        # Lấy nội dung của các cột và lưu vào danh sách dữ liệu
        data.append([column.get_text() for column in columns])

    # Chuyển danh sách dữ liệu thành DataFrame của pandas
    df = pd.DataFrame(data, columns=['Kỳ hạn', 'Other', 'Offline', 'Online'])

    # In ra bảng dữ liệu
    print(df)

else:
    print('Yêu cầu không thành công.')

Yêu cầu không thành công.


2. Lãi suất điều hành SBV

https://www.sbv.gov.vn/webcenter/portal/vi/menu/rm/ls?_afrLoop=34719139833966466#%40%3F_afrLoop%3D34719139833966466%26centerWidth%3D80%2525%26leftWidth%3D20%2525%26rightWidth%3D0%2525%26showFooter%3Dfalse%26showHeader%3Dfalse%26_adf.ctrl-state%3Db21pu33qu_53

In [47]:
SBV_int_url = 'https://www.sbv.gov.vn/webcenter/portal/vi/menu/rm/ls?_afrLoop=34719139833966466#%40%3F_afrLoop%3D34719139833966466%26centerWidth%3D80%2525%26leftWidth%3D20%2525%26rightWidth%3D0%2525%26showFooter%3Dfalse%26showHeader%3Dfalse%26_adf.ctrl-state%3Db21pu33qu_53'
response = requests.get(SBV_int_url)
allsoup = BeautifulSoup(response.text, 'html') 
# Find all rows within the tbody
rows = allsoup.find_all('tr')

# Initialize a list to store the scraped data
data = []

# Refinancing rate
for row in rows:
    cells = row.find_all('td')
    if cells:
        row_data = [cell.get_text(strip=True) for cell in cells]
        data.append(row_data)

Refinancing_tb = pd.DataFrame(data[6:8])
InterBank_tb = pd.DataFrame(data[14:21]).concat('Date':data[12])

2. Quyét dữ liệu lịch sử Lãi suất điều hành của SBV

In [2]:
import warnings
import pandas as pd

# Suppress the openpyxl warning
warnings.filterwarnings("ignore")


theframe = pd.DataFrame(columns=['update_date','Qua đêm','1 Tuần','2 Tuần','1 Tháng','3 Tháng','6 Tháng','9 Tháng'])

for x in range (0,25):
    df = pd.read_excel(f"/Users/nguyenhien/Desktop/Interest rate/Macro Economics/LAI_SUAT_TTLNH ({x}).xlsx")
    new_line = [df.iloc[1,1],df.iloc[5,5],df.iloc[6,5],df.iloc[7,5],df.iloc[8,5],df.iloc[9,5],df.iloc[10,5],df.iloc[11,5]]
    new_row = pd.Series(new_line, index=theframe.columns)
    theframe = pd.concat([theframe, new_row.to_frame().T], ignore_index=True)

warnings.filterwarnings("default")


In [3]:
theframe.to_excel("/Users/nguyenhien/Desktop/Interest rate/Macro Economics/LAI_SUAT_TTLNH_Tổng hợp.xlsx")